# Functions

## Import libraries

In [ ]:
import itertools
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize

## Data preparation

In [ ]:
def prepare_data_q1():
    '''
    This function downloads and prepares the data for the question 1.

    Since the data was originally uploaded on the Classroom page of the course
    and Colab is not able to access it, the .csv file has been re-uploaded
    unaltered to the Google Drive of my student account and made public for
    Colab to access.

    For the train-test split it has been choosen the classic 80/20 split and the
    random state has been fixed to ensure reproducibility of the results.
    '''

    # download age prediction data (flag -q used to avoid prints)
    !gdown 1mOTd6ZjyILyRPqhqi3dgXWSKBwBiNvdu -q

    # load data into Pandas
    df = pd.read_csv('/content/AGE_PREDICTION.csv')

    # convert data to numpy data structures
    x = df.iloc[:, 0 : 32].to_numpy()
    y = df.iloc[:, 32].to_numpy().reshape(-1, 1)

    # split data into training and test sets
    # validation is not split here as we will use k-fold cross-validation
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

    # normalize data
    scaler = StandardScaler()

    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)

    return x_train, x_test, y_train, y_test

## Auxiliary functions

### Losses and errors

In [ ]:
def non_regularized_loss(y_pred, y_true):
    '''
    This function implements the non-regularized loss function that is
    required for the report. It is just a mean square error loss and implements
    the following formula:

    MSE = 1 / N * \sum_{i = 1}^{N} (y_i - y_i\hat)^2
    '''

    return (1.0 / len(y_true)) * np.sum((y_true - y_pred) ** 2)

def l2_loss(y_pred, y_true, lam, weights):
    '''
    This function implements the L2 loss function. This is the main loss
    of the neural network, our objective function. It is the sum of the mean
    square error loss and the L2 regularization term, which is defined as:

    \lam * \sum_{i = 1}^{L} \sum_{j = 1}^{N_i} w_{ij}^2
    '''

    # mean square error component of the loss
    mse = (1.0 / len(y_true)) * np.sum((y_true - y_pred) ** 2)

    # regularization component of the loss
    reg = lam * np.sum([np.sum(omega ** 2) for omega in weights])

    return mse + reg

def mape(y_true, y_pred):
    '''
    This function implements the Mean Absolute Percentage Error. This is the
    metric used to evaluate the performances of the neural network. It
    implements the formula:

    MAPE = 100 / N * \sum_{i = 1}^{N} \frac{|y_i - y_i\hat|}{y_i}
    '''

    return (100 / len(y_true)) * np.sum(np.abs((y_true - y_pred) / y_true))

### Activation functions and derivatives

In [ ]:
def sigmoid(x):
    '''
    Sigmoid function. We clip the input to avoid overflow of the exponential.
    '''

    x = np.clip(x, -500, 500)
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_der(x):
    '''
    Derivative of the sigmoid function.
    '''

    sig = sigmoid(x)

    return sig * (1.0 - sig)

def tanh(x):
    '''
    Hyperbolic tangent function.
    '''

    return np.tanh(x)

def tanh_der(x):
    '''
    Derivative of the hyperbolic tangent function.
    '''

    tanh = np.tanh(x)

    return 1.0 - np.square(tanh)

def swish(x):
    '''
    1-Swish function.
    '''

    return x * sigmoid(x)

def swish_der(x):
    '''
    Derivative of the 1-Swish function.
    '''

    sig = sigmoid(x)

    return sig + x * sig * (1 - sig)

### Initialization

In [ ]:
def xavier_normal_init(in_dim, out_dim):
    '''
    This function implements the Xavier normal initialization for the weights.
    '''

    # standard deviation of xavier normal initialization
    sigma = np.sqrt(2.0 / (in_dim + out_dim))

    return np.random.randn(in_dim, out_dim) * sigma

## Question 1

### Multi-Layer Perceptron

In [ ]:
class MLP():
    '''
    This class implements a Multi-Layer Perceptron neural network for the
    regression task. The neural network is implemented from scratch from numpy
    structures. In particular the forward pass and backpropagation are
    implemented following the formulas presented in the report.

    The training of the neural network is done using scipy.optimize.minimize()
    method.
    '''

    def __init__(self, L, N, lam, activation_name, input_size=32):
        '''
        This function initializes the Multi-Layer Perceptron object. It sets
        the parameters of the neural network and initializes the weights and
        biases.

        The weights are initialized using the Xavier's initialization.
        The biases are initialized to zeros.

        The input size is defaulted to 32 since it is the number of features of
        the provided dataset.
        '''

        # object params initialization
        self.L = L
        self.N = N
        self.lam = lam
        self.input_size = input_size

        # we get the activation function and its derivative from the names of
        # the functions which are defined in cells above
        self.activation = globals().get(activation_name)
        self.activation_der = globals().get(activation_name + "_der")

        # initialize lists of weights and biases
        self.weights = []
        self.biases = []

        # weights and biases initialization:
        # we will use xavier's initialization for the weights
        # and an zeros for the biases

        # from input layer to first hidden layer
        self.weights.append(xavier_normal_init(self.input_size, N[0]))
        self.biases.append(np.zeros(N[0]))

        # between hidden layers
        for i in range(1, L):
            self.weights.append(xavier_normal_init(N[i - 1], N[i]))
            self.biases.append(np.zeros(N[i]))

        # from last hidden layer to output layer
        self.weights.append(xavier_normal_init(N[-1], 1))
        self.biases.append(np.zeros(1))

    def convert_params(self, weights = None, biases = None):
        '''
        This function transforms the weights matrices and biases vectors into a
        single vector. This is needed because the scipy.optimize.minimize()
        function requires a vector as input of the function to optimize.

        We will therefore need to convert back and forth between those two
        representations during training
        '''

        # if we pass no parameters we use the values of the object
        if weights is None and biases is None:
            weights = self.weights
            biases = self.biases

        # convert all weights to single flat vector
        flat_weights = np.concatenate([w.flatten() for w in weights])

        # convert all biases to single flat vector
        flat_biases = np.concatenate([b.flatten() for b in biases])

        # merge the two vectors and return the complete parameters vector
        return np.concatenate([flat_weights, flat_biases])

    def update_params(self, new_params):
        '''
        This function takes as input the vector of all weights and biases
        provided by the scipy.optimize.minimize() function and transforms it
        into the weights matrices and biases vectors of the MLP object.

        It exploits the fact that the positions of weights and biases are
        fixed inside the new_params vector to take chunks of the vector and
        reshape them in the correct forms.
        '''

        offset = 0

        # from input layer to first hidden layers
        self.weights[0] = new_params[offset : offset + self.input_size * self.N[0]].reshape(self.input_size, self.N[0])
        offset += self.input_size * self.N[0]

        # between hidden layers
        for i in range(1, self.L):
            self.weights[i] = new_params[offset : offset + self.N[i - 1] * self.N[i]].reshape(self.N[i - 1], self.N[i])
            offset += self.N[i - 1] * self.N[i]

        # from last hidden layer to output layer
        self.weights[self.L] = new_params[offset : offset + self.N[-1]].reshape(self.N[-1], 1)
        offset += self.N[-1]

        # from input layer to first hidden layers
        self.biases[0] = new_params[offset : offset + self.N[0]]
        offset += self.N[0]

        # between hidden layers
        for i in range(1, self.L):
            self.biases[i] = new_params[offset : offset + self.N[i]]
            offset += self.N[i]

        # from input layer to first hidden layers
        self.biases[-1] = new_params[offset : offset + 1]

    def get_loss(self, x, y):
        '''
        This function computes the predictions and then the loss between the
        ground truth and the predicted values.
        '''

        y_pred = self.predict(x)

        return l2_loss(y_pred, y, self.lam, self.weights)

    def get_non_regularized_loss(self, x, y):
        '''
        This function is the same as the one above but uses the non-regularized
        loss as printing its value is a requirement of the homework.
        '''

        y_pred = self.predict(x)

        return non_regularized_loss(y_pred, y)

    def objective(self, new_params, x, y):
        '''
        This function converts the loss into a function that can be used by the
        scipy.optimize.minimize() method as the 'fun' callable, the objective
        function to be minimized.

        The first argument is the vector of values the minimizer is optimizing
        on, the vector of weights and biases. We first update the weights and
        biases and then compute the loss on the train set.
        '''

        self.update_params(new_params)

        return self.get_loss(x, y)

    def forward_pass(self, x):
        '''
        This function implements the forward pass of the neural network.

        The formulas used here are derived and justified in the report.

        Following the structure of the neural network we apply dot products
        with weight matrices and activations functions and then sum the bias
        until we arrive to the output layer. What we obtain at the end is the
        prediction of the neural network.

        We memorize the values at each step since they are needed for the
        backpropagation.

        We call Z what we obtain after a dot product and we call A what we
        obtain after applying the activation function, following the same
        nomenclature used in the report.
        '''

        # preallocate memory for Z and A
        Z = np.array([None] * self.L)
        A = np.array([None] * self.L)

        # input layer
        Z[0] = np.dot(x, self.weights[0]) + self.biases[0]
        A[0] = self.activation(Z[0])

        # hidden layers
        for i in range(1, self.L):
            Z[i] = np.dot(A[i - 1], self.weights[i]) + self.biases[i]
            A[i] = self.activation(Z[i])

        # output layer
        y_pred = np.dot(A[self.L - 1], self.weights[self.L]) + self.biases[self.L]

        return Z, A, y_pred

    def backpropagation(self, x, y_true):
        '''
        This function implements the backpropagation of the neural network.

        The formulas used here are derived and justified in the report.

        The fact that we are using an L2-regularized loss instead of just the
        MSE means that when we are computing the derivative of the error with
        respect to the weights we need to add the derivative of the
        regularization term, that for L2 are simply the weights themselves.

        For the other components the backpropagation behaves the same as a
        non-regularized MSE loss.

        We call Z what we obtain after a dot product and we call A what we
        obtain after applying the activation function, following the same
        nomenclature used in the report.
        '''

        # backpropagate through all layers and compute the gradients

        n_samples = x.shape[0]
        Z, A, y_pred = self.forward_pass(x)

        # preallocate memory for the gradients
        dEdW = np.array([None] * len(self.weights))
        dEdB = np.array([None] * len(self.biases))

        # OUTPUT LAYER

        dEdY = - 2 * (y_true - y_pred) # = dEdAL = dEdZL since no activation function in the last layer

        dEdW[-1] = np.dot(A[-1].T, dEdY) / n_samples + 2 * self.lam * self.weights[-1]
        dEdB[-1] = np.sum(dEdY) / n_samples

        # HIDDEN LAYERS

        dEdA = np.dot(dEdY, self.weights[-1].T)

        for i in reversed(range(1, self.L)):
            dEdZ = dEdA * self.activation_der(Z[i])

            dEdW[i] = np.dot(A[i - 1].T, dEdZ) / n_samples + 2 * self.lam * self.weights[i]

            dEdB[i] = np.sum(dEdZ, axis = 0) / n_samples

            dEdA = np.dot(dEdZ, self.weights[i].T)

        # INPUT LAYER

        dEdZ = dEdA * self.activation_der(Z[0])

        dEdW[0] = np.dot(x.T, dEdZ) / n_samples + 2 * self.lam * self.weights[0]
        dEdB[0] = np.sum(dEdZ, axis=0) / n_samples

        return dEdW, dEdB

    def backpropagation_opt(self, new_params, x, y):
        '''
        This function converts the backpropagation operation into a function
        that can be used by the scipy.optimize.minimize() method as the 'jac'
        callable, the method for computing the gradient vector.

        As for the objective the first argument is the vector of values the
        minimizer is optimizing on, the vector of weights and biases. We first
        update the weights and biases and then compute the backpropagation.

        After the backpropagation we convert the gradients into the format
        required by the scipy.optimize.minimize().
        '''

        self.update_params(new_params)

        dEdW, dEdB = self.backpropagation(x, y)

        return self.convert_params(dEdW, dEdB)

    def predict(self, x):
        '''
        This function predicts the output of the neural network for the given
        input. It executes the forward pass but keeps only the last element,
        which is the prediction.
        '''

        return self.forward_pass(x)[-1]

    def evaluate(self, x, y):
        '''
        This function evaluates the MAPE  on given input data.
        '''

        y_pred = self.predict(x)

        return mape(y, y_pred)

    def train(self, x, y, verbose = False):
        '''
        This function implements the training of the neural network using the
        scipy.optimize.minimize() method. Optimization time is measured as it
        is required by the homework.
        '''

        # start measuring time
        time_init = time.process_time()

        # solve optimization problem
        result = minimize(fun = self.objective,
                          x0 = self.convert_params(),
                          jac = self.backpropagation_opt,
                          args = (x, y),
                          method = 'L-BFGS-B',
                          options = {"maxiter" : 1000,
                                     "maxls" : 20,
                                     "ftol" : 1e-4})

        # stop measuring time and compute CPU time of optimization
        time_end = time.process_time()
        time_delta = time_end - time_init

        # update weights and biases to optimal value found
        self.update_params(result.x)

        if verbose:
            return result, time_delta
        else:
            return result

### Cross-validation

In [ ]:
def MLP_cross_validation(network_structures, lambdas, activations, k, show_progress = True):
    '''
    This function implements the cross-validation for the MLP hyperparameters.
    The hyperparameters we work on are:
    - The number of layers
    - The number of neurons per layer
    - The activation function
    - The regularization parameter lambda

    Search space is inputted to the method.
    '''

    # load data question 1
    x, _, y, _ = prepare_data_q1()

    # generate all possible combinations
    combinations = list(itertools.product(network_structures, lambdas, activations))

    # split train data into folds
    k_folds = KFold(n_splits = k)

    # initialize variables to find best combination
    best_val_MAPE = float('inf')
    best_train_MAPE = None
    best_val_error = None
    best_train_error = None
    best_combination = None

    for combination in tqdm(combinations) if show_progress else combinations:
        fold_val_MAPE = []
        fold_train_MAPE = []
        fold_val_error = []
        fold_train_error = []

        for train_index, val_index in k_folds.split(x):
            # split data into train and validation sets
            x_train = x[train_index]
            y_train = y[train_index]

            x_val = x[val_index]
            y_val = y[val_index]

            # initialize and train model
            np.random.seed(42)
            model = MLP(L = len(combination[0]),
                        N = combination[0],
                        lam = combination[1],
                        activation_name = combination[2])
            model.train(x_train, y_train)

            # evaluate model
            fold_val_MAPE.append(model.evaluate(x_val, y_val))
            fold_train_MAPE.append(model.evaluate(x_train, y_train))
            fold_val_error.append(model.get_loss(x_val, y_val))
            fold_train_error.append(model.get_loss(x_train, y_train))

        # compute combination MAPE as the mean of the MAPE for all folds
        mean_val_MAPE = np.mean(fold_val_MAPE)
        mean_train_MAPE = np.mean(fold_train_MAPE)
        mean_val_error = np.mean(fold_val_error)
        mean_train_error = np.mean(fold_train_error)

        # update best combination if current combination is better
        if mean_val_MAPE < best_val_MAPE:
            best_val_MAPE = mean_val_MAPE
            best_train_MAPE = mean_train_MAPE
            best_val_error = mean_val_error
            best_train_error = mean_train_error
            best_combination = combination

    return best_combination, best_val_MAPE, best_train_MAPE, best_val_error, best_train_error

## Required prints and values to compile the report

In [ ]:
def MLP_print(L, N, lam, activation):
    '''
    This function implements the training of the MLP and prints all the
    informations required by the homework text.
    '''

    # load data question 1
    x_train, x_test, y_train, y_test = prepare_data_q1()

    # initialize and train model
    np.random.seed(42)
    model = MLP(L, N, lam, activation)
    result, time_delta = model.train(x_train, y_train, verbose = True)

    # prints
    print(f"1. Number of layers L chosen: {L}")
    print(f"2. Number of neurons N chosen: {N}")
    print(f"3. Value of λ chosen: {lam}")
    print(f"4. Values of other hyperparameters (activation function): {activation}")
    print(f"5. Optimization solver chosen: L-BFGS-B")
    print(f"6. Number of iterations: {result.nit}")
    print(f"7. Time for optimizing the network (from when the solver is called until it stops): {time_delta} seconds")
    print(f"8. Training Error (defined as in question (1), without regularization term): {model.get_non_regularized_loss(x_train, y_train)}")
    print(f"9. Test Error (defined as above but on the test set): {model.get_non_regularized_loss(x_test, y_test)}")

In [ ]:
def MLP_report(L, N, lam, activation):
    '''
    This function implements the training of the MLP and prints all the
    informations to be inserted into the report.
    '''

    # The final setting for selected non-linearity, L, Nl, and λ
    print(f"Final setting for selected non-linearity, L, Nl, and λ:")
    print(f"\tNon-linearity: {activation}")
    print(f"\tL: {L}")
    print(f"\tNl: {N}")
    print(f"\tλ: {lam}")

    # which optimization routine you used for solving the minimization problem,
    # the setting of its parameters (max number of iterations, etc.) and the
    # returned message in output (successful optimization or others, number of
    # iterations, starting/final value of the objective function) if any;
    print(f"Optimization routine used: scipy.optimize.minimize() with the solver L-BFGS-B")
    print(f"Setting of its parameters:")
    print(f"\tMax number of iterations: 1000")
    print(f"\tMax number of iterations for linesearch: 20")
    print(f"\tTolerance on the objective function: 1e-4")

    # load data question 1
    x_train, x_test, y_train, y_test = prepare_data_q1()

    # initialize model
    np.random.seed(42)
    model = MLP(L, N, lam, activation)
    initial_error = model.get_loss(x_train, y_train)
    initial_MAPE = model.evaluate(x_train, y_train)

    # train model
    result, time_delta = model.train(x_train, y_train, verbose = True)

    print(f"Output of the optimizer:\n{result}")

    print(f"Initial value of the regularized error on the training set: {initial_error}")

    print(f"Final value of the regularized error on the training set: {model.get_loss(x_train, y_train)}")

    # the value of the error on the validation set (avg of errors on the k-folds)
    _, kfold_val_MAPE, kfold_train_MAPE, kfold_val_error, kfold_train_error = MLP_cross_validation([N], [lam], [activation], k = 5, show_progress = False)
    print(f"Final value of the regularized error on the training set (k-folds): {kfold_train_error}")
    print(f"Final value of the regularized error on the validation set: {kfold_val_error}")

    # the final value of the test error
    print(f"Final value of the test error: {model.get_loss(x_test, y_test)}")

    # the initial and final MAPE (average of MAPE values obtained with k-fold)
    # on the training set
    print(f"Initial MAPE on training set: {initial_MAPE}")
    print(f"Final MAPE on training set: {kfold_train_MAPE}")


    # the final MAPE on validation and test set
    print(f"Final MAPE on validation set: {kfold_val_MAPE}")
    print(f"Final MAPE on test set: {model.evaluate(x_test, y_test)}")

    # TABLE

    # optimization time
    print(f"Optimization time: {time_delta} seconds")

# Execution

In [ ]:
# set mode to hide all non-required prints
hide_non_required_prints = True

In [ ]:
# setup cross-validation space
network_structures = [(8, 4), (16, 8), (32, 16), (64, 32), (16, 8, 4), (32, 16, 8), (64, 32, 16), (16, 8, 4, 2), (32, 16, 8, 4)]
lambdas = [0, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 5e-1, 1e+0]
activations = ["sigmoid", "tanh", "swish"]

# execute cross-validation
best_combination, best_val_MAPE, best_train_MAPE, best_val_error, best_train_error = MLP_cross_validation(network_structures, lambdas, activations, k = 5)

print(f"\nBest hyperparameters: {best_combination}")
# expected result is: ((64, 32), 0.05, 'swish')

100%|██████████| 270/270 [3:48:28<00:00, 50.77s/it]


Best hyperparameters: ((64, 32), 0.05, 'swish')


In [ ]:
# print values that are needed to compile the report but are not required
# to be printed on the notebook

# non-required and hidden
if not hide_non_required_prints:
    MLP_report(L = 2,
               N = (64, 32),
               lam = 0.05,
               activation = 'swish')

In [ ]:
# print values that are required to be printed on the notebook

MLP_print(L = 2,
          N = (64, 32),
          lam = 0.05,
          activation = 'swish')

1. Number of layers L chosen: 2
2. Number of neurons N chosen: (64, 32)
3. Value of λ chosen: 0.05
4. Values of other hyperparameters (activation function): swish
5. Optimization solver chosen: L-BFGS-B
6. Number of iterations: 75
7. Time for optimizing the network (from when the solver is called until it stops): 34.243861524999375 seconds
8. Training Error (defined as in question (1), without regularization term): 96.27197172136378
9. Test Error (defined as above but on the test set): 90.89298342505549
